# Start Real Experiment

A financial institution was facing increasing pressure to improve the efficiency and fairness of its loan approval process.  
With growing competition and a push toward digital transformation, the bank needed to reduce manual decision-making, minimize default risk, and ensure more consistent approvals. However, loan officers were relying heavily on subjective judgment, leading to inconsistencies, delays, and missed opportunities. 

To address this, the bank began collecting detailed data on past loan applicants—including their income, employment status, credit history, and loan outcomes—with the goal of developing a data-driven system to predict loan approval chances. 

Our task is to analyze this data, build a reliable prediction model, and help the bank automate and optimize its loan evaluation process.

## Loan Prediction

In [1]:
# Importing the required packages
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.preprocessing import  LabelEncoder
import mlflow
import os


Setup the user name to be logged

In [2]:
os.environ["LOGNAME"] = "Heba"
RANDOM_SEED = 6

### Load & Explore The Data

In [3]:
# load the dataset
loan_data_path = r"./loan_data.csv"
dataset = pd.read_csv(loan_data_path)
dataset.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


The loan dataset consists of 614 records, each representing a loan application with 13 features capturing applicant details and loan information. 

Key attributes include demographic data such as gender, marital status, and education; financial details like applicant and coapplicant income; and loan-specific information including loan amount, loan term, credit history, and loan status (approved or not). 

While most columns are complete, some contain missing values, particularly in "Credit\_History", "LoanAmount", and "Self\_Employed", indicating the need for preprocessing before analysis or modeling.

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


### **Data Preprocessing Steps**

1. **Feature Separation**

   * Numerical columns were identified using data types `int64` and `float64`.
   * Categorical columns were identified using the `object` type, excluding the target column `Loan_Status` and the identifier `Loan_ID`.

2. **Handling Missing Values**

   * **Categorical columns**: Missing values were filled with the mode (most frequent value) of each column.
   * **Numerical columns**: Missing values were filled with the median of each column.

3. **Outlier Treatment**

   * Numerical columns were clipped to their 5th and 95th percentiles to reduce the impact of extreme outliers.

4. **Feature Transformation and Engineering**

   * Applied log transformation to the `LoanAmount` feature to normalize its distribution.
   * Created a new feature `TotalIncome` by summing `ApplicantIncome` and `CoapplicantIncome`, then applied a log transformation to it.
   * Removed the original `ApplicantIncome` and `CoapplicantIncome` columns as they were now represented in `TotalIncome`.

5. **Encoding Categorical Variables**

   * All categorical columns (excluding `Loan_Status` and `Loan_ID`) were label-encoded using `LabelEncoder`.
   * The target variable `Loan_Status` was also label-encoded to prepare it for classification.

6. **Train-Test Split**

   * The dataset was split into training and testing sets using a 70-30 ratio, with a fixed random seed (`random_state=6`) to ensure reproducibility.
   * Features (`X`) excluded `Loan_Status` and `Loan_ID`, while the target (`y`) was set as `Loan_Status`.

In [5]:
numerical_cols = dataset.select_dtypes(include=['int64','float64']).columns.tolist()
categorical_cols = dataset.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('Loan_Status')
categorical_cols.remove('Loan_ID')

# Filling categorical columns with mode
for col in categorical_cols:
    dataset[col] = dataset[col].fillna(dataset[col].mode()[0])

# Filling Numerical columns with median
for col in numerical_cols:
    dataset[col] = dataset[col].fillna(dataset[col].median())

# Take care of outliers
dataset[numerical_cols] = dataset[numerical_cols].apply(lambda x: x.clip(*x.quantile([0.05, 0.95])))

# Log Transforamtion & Domain Processing
dataset['LoanAmount'] = np.log(dataset['LoanAmount']).copy()
dataset['TotalIncome'] = dataset['ApplicantIncome'] + dataset['CoapplicantIncome']
dataset['TotalIncome'] = np.log(dataset['TotalIncome']).copy()


# Dropping ApplicantIncome and CoapplicantIncome
dataset = dataset.drop(columns=['ApplicantIncome','CoapplicantIncome'])

# Label encoding categorical variables
for col in categorical_cols:
    le = LabelEncoder()
    dataset[col] = le.fit_transform(dataset[col])

#Encode the target columns
dataset['Loan_Status'] = le.fit_transform(dataset['Loan_Status'])

# Train test split
X = dataset.drop(columns=['Loan_Status', 'Loan_ID'])
y = dataset.Loan_Status

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size =0.3, random_state = RANDOM_SEED)

The previous code has several issues and bad practicies:

- Imputation on the whole dataset (data leakage)  
    Problem: You compute modes and medians on all 614 records, then split into train/test—information from the test set leaks into the imputers.

- Outlier clipping before split (more leakage)  
    Problem: Clipping by the 5th/95th percentiles of the full dataset again leaks test information.

- Log-transforming zero or negative values  
    Problem: Taking np.log(…) of zeros (e.g. if LoanAmount or TotalIncome ever equals 0) raises -inf or NaNs.

- Reusing the same LabelEncoder for all columns and the target  
    Problem: You instantiate a new LabelEncoder inside the loop, but then reuse the last one to encode Loan_Status, so the mapping might be inconsistent.

- Pipeline & reproducibility  
    Problem: A manual “for loop” approach is hard to maintain, and easy to get out of sync between train/test.

In [4]:
# Identify column types
numerical_cols = dataset.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = dataset.select_dtypes(include=['object']).columns.tolist()
categorical_cols.remove('Loan_Status')
categorical_cols.remove('Loan_ID')

# Train-test split before any transformation
X = dataset.drop(columns=['Loan_Status', 'Loan_ID'])
y = dataset['Loan_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_SEED)

# === Impute categorical features ===
from sklearn.impute import SimpleImputer

cat_imputer = SimpleImputer(strategy='most_frequent')
X_train[categorical_cols] = cat_imputer.fit_transform(X_train[categorical_cols])
X_test[categorical_cols] = cat_imputer.transform(X_test[categorical_cols])

# === Impute numerical features ===
num_imputer = SimpleImputer(strategy='median')
X_train[numerical_cols] = num_imputer.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = num_imputer.transform(X_test[numerical_cols])

# === Clip outliers (based on train quantiles) ===
lower = X_train[numerical_cols].quantile(0.05)
upper = X_train[numerical_cols].quantile(0.95)
X_train[numerical_cols] = X_train[numerical_cols].clip(lower=lower, upper=upper, axis=1)
X_test[numerical_cols] = X_test[numerical_cols].clip(lower=lower, upper=upper, axis=1)

# === Feature engineering: log(LoanAmount), TotalIncome ===
X_train['LoanAmount'] = np.log1p(X_train['LoanAmount'])
X_test['LoanAmount'] = np.log1p(X_test['LoanAmount'])

X_train['TotalIncome'] = X_train['ApplicantIncome'] + X_train['CoapplicantIncome']
X_test['TotalIncome'] = X_test['ApplicantIncome'] + X_test['CoapplicantIncome']

X_train['TotalIncome'] = np.log1p(X_train['TotalIncome'])
X_test['TotalIncome'] = np.log1p(X_test['TotalIncome'])

# Drop original income columns
X_train = X_train.drop(columns=['ApplicantIncome', 'CoapplicantIncome'])
X_test = X_test.drop(columns=['ApplicantIncome', 'CoapplicantIncome'])

# === Label encode categorical variables ===
cat_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])
    cat_encoders[col] = le  # Save encoder if needed later

# === Encode target variable ===
target_le = LabelEncoder()
y_train = target_le.fit_transform(y_train)
y_test = target_le.transform(y_test)

In [ ]:
# from sklearn.pipeline import Pipeline, make_pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import FunctionTransformer, OrdinalEncoder, LabelEncoder
# from sklearn.base import BaseEstimator, TransformerMixin

# # 1) Split first to avoid leakage
# X = dataset.drop(columns=['Loan_Status', 'Loan_ID'])
# y = dataset['Loan_Status']

# X_train, X_test, y_train, y_test = train_test_split(
#     X, y, test_size=0.3, random_state=6
# )

# # 2) Define column lists
# numerical_cols = X_train.select_dtypes(include=['int64','float64']).columns.tolist()
# categorical_cols = X_train.select_dtypes(include=['object']).columns.tolist()

# # 3.1) Custom transformer for outlier clipping
# class QuantileClipper(BaseEstimator, TransformerMixin):
#     def __init__(self, lower=0.05, upper=0.95):
#         self.lower = lower
#         self.upper = upper

#     def fit(self, X, y=None):
#         if isinstance(X, np.ndarray):
#             X = pd.DataFrame(X)

#         # Compute quantiles on TRAINING data only
#         self.q_lower_ = X.quantile(self.lower)
#         self.q_upper_ = X.quantile(self.upper)
#         return self

#     def transform(self, X):
#         return X.clip(self.q_lower_, self.q_upper_)

# # 3.2) Custom feature engineering transformer
# class TotalIncomeAdder(BaseEstimator, TransformerMixin):
#     def __init__(self, numerical_cols):
#         self.numerical_cols = numerical_cols

#     def fit(self, X, y=None):
#         # Store feature names during fit
#         if isinstance(X, pd.DataFrame):
#             self.feature_names_ = X.columns
#         else:
#             self.feature_names_ = self.numerical_cols # Fallback if it's a NumPy array
#         return self

#     def transform(self, X):
#         # Convert to DataFrame if needed
#         if isinstance(X, np.ndarray):
#             X = pd.DataFrame(X, columns=self.feature_names_)
#         X = X.copy()
#         if all(col in X.columns for col in ['ApplicantIncome', 'CoapplicantIncome']):
#             X['TotalIncome'] = X['ApplicantIncome'] + X['CoapplicantIncome']
#             X = X.drop(columns=['ApplicantIncome', 'CoapplicantIncome'])
#         return X

# # 4) Build numerical pipeline
# num_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='median')),
#     ('clipper', QuantileClipper(lower=0.05, upper=0.95)),
#     ('total_income', TotalIncomeAdder(numerical_cols)),
#     ('log', FunctionTransformer(np.log1p, validate=False))
# ])

# # 5) Build categorical pipeline
# cat_pipeline = Pipeline([
#     ('imputer', SimpleImputer(strategy='most_frequent')),
#     ('encoder', OrdinalEncoder())     # or OneHotEncoder() if you prefer
# ])

# # 6) Combine into a ColumnTransformer
# preprocessor = ColumnTransformer([
#     ('cat', cat_pipeline, categorical_cols),
#     ('num', num_pipeline, numerical_cols)
# ])

# # 7) Fit & transform data
# X_train_proc = preprocessor.fit_transform(X_train)
# X_test_proc  = preprocessor.transform(X_test)

# # 8) Encode target separately
# target_le = LabelEncoder()
# y_train_enc = target_le.fit_transform(y_train)
# y_test_enc  = target_le.transform(y_test)

### Experiment with different models

Now, we'll experiment with three different models [Random Forest Classifier, Logistic Regression, Decision Tree Classifier] using grid search for each

In [5]:
# RandomForest
rf = RandomForestClassifier(random_state=RANDOM_SEED)
param_grid_forest = {
    'n_estimators': [200,400, 700],
    'max_depth': [10,20,30],
    'criterion' : ["gini", "entropy"],
    'max_leaf_nodes': [50, 100]
}

grid_forest = GridSearchCV(
        estimator=rf,
        param_grid=param_grid_forest, 
        cv=5, 
        n_jobs=-1, 
        scoring='accuracy',
        verbose=0
    )
model_forest = grid_forest.fit(X_train, y_train)

#Logistic Regression

lr = LogisticRegression(random_state=RANDOM_SEED)
param_grid_log = {
    'C': [100, 10, 1.0, 0.1, 0.01],
    'penalty': ['l1','l2'],
    'solver':['liblinear']
}

grid_log = GridSearchCV(
        estimator=lr,
        param_grid=param_grid_log, 
        cv=5,
        n_jobs=-1,
        scoring='accuracy',
        verbose=0
    )
model_log = grid_log.fit(X_train, y_train)

#Decision Tree

dt = DecisionTreeClassifier(
    random_state=RANDOM_SEED
)

param_grid_tree = {
    "max_depth": [3, 5, 7, 9, 11, 13],
    'criterion' : ["gini", "entropy"],
}

grid_tree = GridSearchCV(
        estimator=dt,
        param_grid=param_grid_tree, 
        cv=5,
        n_jobs=-1,
        scoring='accuracy',
        verbose=0
    )
model_tree = grid_tree.fit(X_train, y_train)

### Log the experiments

Firstly, set the tracking URI & experiment

In [6]:
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Loan_prediction")

2025/05/10 02:44:20 INFO mlflow.tracking.fluent: Experiment with name 'Loan_prediction' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/517529337940852463', creation_time=1746834260648, experiment_id='517529337940852463', last_update_time=1746834260648, lifecycle_stage='active', name='Loan_prediction', tags={}>

Here, we define a model evaluation and logging process using **MLflow**. 

It starts by implementing a custom evaluation function `eval_metrics` that calculates key performance metrics: accuracy, F1-score, and AUC, and generates a ROC curve which is saved as an image. 

Then, the `mlflow_logging` function takes a trained model and logs its best hyperparameters, evaluation metrics, the ROC curve plot as an artifact, and system metrics. It also logs the model itself and test dataset info for experiment tracking and reproducibility. 

Finally, this logging function is applied to three different models — Decision Tree, Logistic Regression, and Random Forest — to record and compare their performance on the test data.

In [7]:
import mlflow.models
import mlflow.sklearn


def eval_metrics(actual, pred):
    accuracy = metrics.accuracy_score(actual, pred)
    f1 = metrics.f1_score(actual, pred, pos_label=1)
    fpr, tpr, _ = metrics.roc_curve(actual, pred)
    auc = metrics.auc(fpr, tpr)
    plt.figure(figsize=(8,8))
    plt.plot(fpr, tpr, color='blue', label='ROC curve area = %0.2f'%auc)
    plt.plot([0,1],[0,1], 'r--')
    plt.xlim([-0.1, 1.1])
    plt.ylim([-0.1, 1.1])
    plt.xlabel('False Positive Rate', size=14)
    plt.ylabel('True Positive Rate', size=14)
    plt.legend(loc='lower right')
    # Save plot
    os.makedirs("./plots", exist_ok=True)
    plt.savefig("./plots/ROC_curve.png")
    # Close plot
    plt.close()
    return(accuracy, f1, auc)


def mlflow_logging(model, X, y, name):
    
     with mlflow.start_run(run_name=name) as run:
        pred = model.predict(X)
        #metrics
        (accuracy, f1, auc) = eval_metrics(y, pred)
        # Logging best parameters from gridsearch
        mlflow.log_params(model.best_params_)
        #log the metrics
        mlflow.log_metrics({
            "Accuracy": accuracy,
            "f1-score": f1,
            "AUC": auc,
            "Mean CV score": model.best_score_
        })

        # Logging artifacts
        mlflow.log_artifact("./plots/ROC_curve.png")

        ###### LOGGING THE DATASET & MODEL ######
        # Create an instance of a PandasDataset

        signature = mlflow.models.infer_signature(X, y) ### INFERING THE SIGNATURE
        mlflow.sklearn.log_model(model, name, signature=signature, input_example=X) ### LOGGING THE MODEL

        dataset = mlflow.data.from_pandas(
            X, name="Loan Data Testing"
        )
        mlflow.log_input(dataset, context="testing")
    
        

mlflow_logging(model_tree, X_test, y_test, "DecisionTreeClassifier")
mlflow_logging(model_log, X_test, y_test, "LogisticRegression")
mlflow_logging(model_forest, X_test, y_test, "RandomForestClassifier")

c:\Users\RTX\miniconda3\envs\mlops-course\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\RTX\miniconda3\envs\mlops-course\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data 

🏃 View run DecisionTreeClassifier at: http://localhost:5000/#/experiments/517529337940852463/runs/022e352fafe64c678da1a4632b5ed623
🧪 View experiment at: http://localhost:5000/#/experiments/517529337940852463


c:\Users\RTX\miniconda3\envs\mlops-course\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\RTX\miniconda3\envs\mlops-course\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data 

🏃 View run LogisticRegression at: http://localhost:5000/#/experiments/517529337940852463/runs/7f296bfbda7f43848803f5c9e062389d
🧪 View experiment at: http://localhost:5000/#/experiments/517529337940852463


c:\Users\RTX\miniconda3\envs\mlops-course\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


🏃 View run RandomForestClassifier at: http://localhost:5000/#/experiments/517529337940852463/runs/040e54fc1d1e4346ad8040b08ccb599b
🧪 View experiment at: http://localhost:5000/#/experiments/517529337940852463


c:\Users\RTX\miniconda3\envs\mlops-course\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


## Nested Runs

Here, we'll use a parent run with different nested runs representing the grid search training process.

Introduced in this experimet, the `mlflow.evaluate`

In [ ]:
# Start a parent run
with mlflow.start_run(run_name="RandomForest Parent Experiment") as parent_run:
    parent_run_id = parent_run.info.run_id  # Get the parent run ID
    # Create an instance of a PandasDataset
    dataset = mlflow.data.from_pandas(
        X_train, name="Loan Data Training"
    )

    mlflow.log_input(dataset, context="training")
    
    # Start a nested run for RandomForest
    with mlflow.start_run(run_name="RandomForest Experiment", nested=True):
        rf = RandomForestClassifier(random_state=RANDOM_SEED)
        param_grid_forest = {
            'n_estimators': [200, 400, 700],
            'max_depth': [10, 20, 30],
            'criterion': ["gini", "entropy"],
            'max_leaf_nodes': [50, 100]
        }

        grid_forest = GridSearchCV(
            estimator=rf,
            param_grid=param_grid_forest,
            cv=5,
            n_jobs=-1,
            scoring='accuracy',
            verbose=0
        )
        model_forest = grid_forest.fit(X_train, y_train)

        # Log parameters and metrics for the nested run
        mlflow.log_params(grid_forest.best_params_)
        # mlflow.log_params(model_forest.best_params_)
        mlflow.log_metric("Mean CV score", grid_forest.best_score_)

        # Log the model
        mlflow.sklearn.log_model(model_forest, "RandomForestModel")

        ######## Evaluation Logging ########
        eval_data = X_test.copy()
        eval_data['Loan_Status'] = y_test
        # Assign the decoded predictions to the Evaluation Dataset
        eval_data["Predictions"] = model_forest.predict(X=X_test)

        # Create the PandasDataset for use in mlflow evaluate
        pd_dataset = mlflow.data.from_pandas(
            eval_data, predictions="Predictions", targets="Loan_Status", name="Loan Data Evaluation"
        )

        result = mlflow.evaluate(data=pd_dataset, predictions=None, model_type="classifier")

c:\Users\RTX\miniconda3\envs\mlops-course\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/05/09 02:10:17 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
c:\Users\RTX\miniconda3\envs\mlo

🏃 View run RandomForest Experiment at: http://localhost:5000/#/experiments/337955864653929902/runs/8905d706ac7946e6a5c6760ac4068a72
🧪 View experiment at: http://localhost:5000/#/experiments/337955864653929902
🏃 View run RandomForest Parent Experiment at: http://localhost:5000/#/experiments/337955864653929902/runs/a72a811299f6493eae29c4b302836377
🧪 View experiment at: http://localhost:5000/#/experiments/337955864653929902


<Figure size 1050x700 with 0 Axes>

Here, we will log each step of the training of the grid search process.

In [ ]:
from sklearn.model_selection import GridSearchCV
import mlflow

# Function to log each grid search iteration
def log_grid_search_results(grid_search, model_name):
    """
    Logs all grid search results into MLflow.
    """
    with mlflow.start_run(run_name=f"{model_name} Grid Search"):
        # Log all hyperparameter combinations and their scores
        # Create an instance of a PandasDataset
        dataset = mlflow.data.from_pandas(
            X_train, name="Loan Data Training"
        )

        mlflow.log_input(dataset, context="training")

        for i in range(len(grid_search.cv_results_['params'])):
            params = grid_search.cv_results_['params'][i]
            mean_test_score = grid_search.cv_results_['mean_test_score'][i]
            std_test_score = grid_search.cv_results_['std_test_score'][i]
            
            # Start a nested run for each hyperparameter combination
            with mlflow.start_run(run_name=f"Params: {params}", nested=True):
                mlflow.log_params(params)
                mlflow.log_metrics({
                    "Mean Test Score": mean_test_score,
                    "Std Test Score": std_test_score
                })
                ######## Evaluation Logging ########
                eval_data = X_test.copy()
                eval_data['Loan_Status'] = y_test
                # Assign the decoded predictions to the Evaluation Dataset
                eval_data["Predictions"] = grid_search.predict(X=X_test)

                # Create the PandasDataset for use in mlflow evaluate
                pd_dataset = mlflow.data.from_pandas(
                    eval_data, predictions="Predictions", targets="Loan_Status", name="Loan Data Evaluation"
                )
                
                signature = mlflow.models.infer_signature(X_train, y_train) ### INFERING THE SIGNATURE
                mlflow.sklearn.log_model(model_forest, "RandomForestModel", signature=signature, input_example=X_train.iloc[[0]])

                result = mlflow.evaluate(data=pd_dataset, predictions=None, model_type="classifier")

        # Log the best parameters and score
        mlflow.log_params(grid_search.best_params_)
        mlflow.log_metric("Best Mean Test Score", grid_search.best_score_)
        

# Example: Random Forest Grid Search
rf = RandomForestClassifier(random_state=RANDOM_SEED)
param_grid_forest = {
    'n_estimators': [200, 400, 700],
    'max_depth': [10, 20, 30],
    'criterion': ["gini", "entropy"],
    'max_leaf_nodes': [50, 100]
}

grid_forest = GridSearchCV(
    estimator=rf,
    param_grid=param_grid_forest,
    cv=5,
    n_jobs=-1,
    scoring='accuracy',
    verbose=0
)

# Fit the grid search
grid_forest.fit(X_train, y_train)

# Log all grid search results into MLflow
log_grid_search_results(grid_forest, "RandomForestClassifier")

c:\Users\RTX\miniconda3\envs\mlops-course\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
c:\Users\RTX\miniconda3\envs\mlops-course\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data 

🏃 View run Params: {'criterion': 'gini', 'max_depth': 10, 'max_leaf_nodes': 50, 'n_estimators': 200} at: http://localhost:5000/#/experiments/517529337940852463/runs/69091998f63747d49e572666a8c7bd4d
🧪 View experiment at: http://localhost:5000/#/experiments/517529337940852463


c:\Users\RTX\miniconda3\envs\mlops-course\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/05/10 03:40:27 WARNING mlflow.models.evaluation.evaluators.classifier: According to the evaluation dataset label values, the model type looks like None, but you specified model type 'classifier'. Please verify that you set 

🏃 View run Params: {'criterion': 'gini', 'max_depth': 10, 'max_leaf_nodes': 50, 'n_estimators': 400} at: http://localhost:5000/#/experiments/517529337940852463/runs/d048c6b3e6684573b53a930029f26faa
🧪 View experiment at: http://localhost:5000/#/experiments/517529337940852463


c:\Users\RTX\miniconda3\envs\mlops-course\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/05/10 03:40:33 WARNING mlflow.models.evaluation.evaluators.classifier: According to the evaluation dataset label values, the model type looks like None, but you specified model type 'classifier'. Please verify that you set 

...

c:\Users\RTX\miniconda3\envs\mlops-course\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/05/10 03:43:40 WARNING mlflow.models.evaluation.evaluators.classifier: According to the evaluation dataset label values, the model type looks like None, but you specified model type 'classifier'. Please verify that you set 

🏃 View run Params: {'criterion': 'entropy', 'max_depth': 30, 'max_leaf_nodes': 100, 'n_estimators': 400} at: http://localhost:5000/#/experiments/517529337940852463/runs/e57ae7aaec484fdb8bd598b62c102be3
🧪 View experiment at: http://localhost:5000/#/experiments/517529337940852463


c:\Users\RTX\miniconda3\envs\mlops-course\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(
2025/05/10 03:43:46 WARNING mlflow.models.evaluation.evaluators.classifier: According to the evaluation dataset label values, the model type looks like None, but you specified model type 'classifier'. Please verify that you set 

🏃 View run Params: {'criterion': 'entropy', 'max_depth': 30, 'max_leaf_nodes': 100, 'n_estimators': 700} at: http://localhost:5000/#/experiments/517529337940852463/runs/c1529ce5797643e2a5bd5529b3d0daeb
🧪 View experiment at: http://localhost:5000/#/experiments/517529337940852463
🏃 View run RandomForestClassifier Grid Search at: http://localhost:5000/#/experiments/517529337940852463/runs/e343f792a0f24632b9c6daf03943df14
🧪 View experiment at: http://localhost:5000/#/experiments/517529337940852463


<Figure size 1050x700 with 0 Axes>

### Load Data

If you only logged metadata using mlflow.data.from_pandas():

This logs metadata only: schema, shape, etc.

The actual data is not saved, so you cannot load it back later.

This is for lineage/tracking, not storage.

In [16]:
# Retrieve the run information
logged_run = mlflow.get_run("e343f792a0f24632b9c6daf03943df14")

# Retrieve the Dataset object
logged_dataset = logged_run.inputs.dataset_inputs[0].dataset

# View some of the recorded Dataset information
print(f"Dataset name: {logged_dataset.name}")
print(f"Dataset digest: {logged_dataset.digest}")
print(f"Dataset profile: {logged_dataset.profile}")
print(f"Dataset schema: {logged_dataset.schema}")

Dataset name: Loan Data Training
Dataset digest: 27d345db
Dataset profile: {"num_rows": 429, "num_elements": 4290}
Dataset schema: {"mlflow_colspec": [{"type": "long", "name": "Gender", "required": true}, {"type": "long", "name": "Married", "required": true}, {"type": "long", "name": "Dependents", "required": true}, {"type": "long", "name": "Education", "required": true}, {"type": "long", "name": "Self_Employed", "required": true}, {"type": "double", "name": "LoanAmount", "required": true}, {"type": "double", "name": "Loan_Amount_Term", "required": true}, {"type": "double", "name": "Credit_History", "required": true}, {"type": "long", "name": "Property_Area", "required": true}, {"type": "double", "name": "TotalIncome", "required": true}]}


In [17]:
# Loading the dataset's source
dataset_source = mlflow.data.get_source(logged_dataset)

local_dataset = dataset_source.load()

print(f"The local file where the data has been downloaded to: {local_dataset}")

# Load the data again
loaded_data = pd.read_csv(local_dataset, delimiter=",")

loaded_data.head()

NotImplementedError: 

### Load Model

In MLflow, models that are logged during training are saved as artifacts, allowing for easy reuse and deployment. These logged models can later be loaded either from the local directory where they were saved or directly from a specific run’s artifact URI. This makes it simple to reproduce results, compare models, or integrate them into production workflows without retraining.

Here we have the original model and we'll compare the model output to the logged model output

In [18]:
dt.predict(pd.DataFrame(X_test))

NotFittedError: This DecisionTreeClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [30]:
logged_model = 'runs:/b3848d6bfcf54499951322f4b5984e5e/DecisionTreeClassifier'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
loaded_model.predict(pd.DataFrame(X_test))

array([0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1])

In [8]:
!mlflow models serve -m mlflow-artifacts:/616713827796020567/ebfcc53f9c914f449567b82846e71dc7/artifacts/LogisticRegression --port 9000                            

:: [Info] ::  Mirror: https://www.python.org/ftp/python
pyenv-install: definition not found: 3.12.9

See all available versions with `pyenv install --list'.


2025/04/30 01:21:49 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2025/04/30 01:21:53 INFO mlflow.utils.virtualenv: Installing python 3.12.9 if it does not exist
Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\RTX\miniconda3\envs\mlops-course\Scripts\mlflow.exe\__main__.py", line 7, in <module>
  File "C:\Users\RTX\miniconda3\envs\mlops-course\Lib\site-packages\click\core.py", line 1161, in __call__
    return self.main(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\RTX\miniconda3\envs\mlops-course\Lib\site-packages\click\core.py", line 1082, in main
    rv = self.invoke(ctx)
         ^^^^^^^^^^^^^^^^
  File "C:\Users\RTX\miniconda3\envs\mlops-course\Lib\site-packages\click\core.py", line 1697, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
                           ^^^^^^^^^^^^^^^^^^^^^

## Auto Logging

This will enable MLflow to automatically log various information about your run, including:
- Metrics - MLflow pre-selects a set of metrics to log, based on what model and library you use
- Parameters - hyper params specified for the training, plus default values provided by the library if not explicitly set
- Model Signature - logs Model signature instance, which describes input and output schema of the model
- Artifacts - e.g. model checkpoints
- Dataset - dataset object used for training (if applicable), such as tensorflow.data.Dataset


In [19]:
mlflow.set_experiment("Loan_prediction_2")

2025/05/10 03:46:12 INFO mlflow.tracking.fluent: Experiment with name 'Loan_prediction_2' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/760520234981114590', creation_time=1746837972837, experiment_id='760520234981114590', last_update_time=1746837972837, lifecycle_stage='active', name='Loan_prediction_2', tags={}>

In [20]:
mlflow.autolog()

## Customizing autologging
# mlflow.autolog(
#     log_model_signatures=False,
#     extra_tags={"YOUR_TAG": "VALUE"},
# )



# RandomForest
rf = RandomForestClassifier(random_state=RANDOM_SEED)
param_grid_forest = {
    'n_estimators': [200,400, 700],
    'max_depth': [10,20,30],
    'criterion' : ["gini", "entropy"],
    'max_leaf_nodes': [50, 100]
}

grid_forest = GridSearchCV(
        estimator=rf,
        param_grid=param_grid_forest, 
        cv=5, 
        n_jobs=-1, 
        scoring='accuracy',
        verbose=0
    )
model_forest = grid_forest.fit(X_train, y_train)

#Logistic Regression

lr = LogisticRegression(random_state=RANDOM_SEED)
param_grid_log = {
    'C': [100, 10, 1.0, 0.1, 0.01],
    'penalty': ['l1','l2'],
    'solver':['liblinear']
}

grid_log = GridSearchCV(
        estimator=lr,
        param_grid=param_grid_log, 
        cv=5,
        n_jobs=-1,
        scoring='accuracy',
        verbose=0
    )
model_log = grid_log.fit(X_train, y_train)

#Decision Tree

dt = DecisionTreeClassifier(
    random_state=RANDOM_SEED
)

param_grid_tree = {
    "max_depth": [3, 5, 7, 9, 11, 13],
    'criterion' : ["gini", "entropy"],
}

grid_tree = GridSearchCV(
        estimator=dt,
        param_grid=param_grid_tree, 
        cv=5,
        n_jobs=-1,
        scoring='accuracy',
        verbose=0
    )
model_tree = grid_tree.fit(X_train, y_train)


2025/05/10 03:46:13 INFO mlflow.tracking.fluent: Autologging successfully enabled for sklearn.
2025/05/10 03:46:13 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '26d5c0eb393f4fbb85c6b9bd843fc92e', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2025/05/10 03:46:13 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\RTX\miniconda3\envs\mlops-course\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as do

🏃 View run wise-ray-823 at: http://localhost:5000/#/experiments/760520234981114590/runs/107f6c8293d94006aa4708794ec6fe9c
🧪 View experiment at: http://localhost:5000/#/experiments/760520234981114590


2025/05/10 03:46:45 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '3eb4caefd9934d5bb9d2fd9be5631112', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow


🏃 View run efficient-dove-660 at: http://localhost:5000/#/experiments/760520234981114590/runs/5cf55769dea3449e8a83aea777bf6fed
🧪 View experiment at: http://localhost:5000/#/experiments/760520234981114590
🏃 View run luxuriant-snake-756 at: http://localhost:5000/#/experiments/760520234981114590/runs/864433e97d5e435fa3a17bffec027cb5
🧪 View experiment at: http://localhost:5000/#/experiments/760520234981114590
🏃 View run welcoming-chimp-378 at: http://localhost:5000/#/experiments/760520234981114590/runs/a24b094e792942f7a5141379b29cd712
🧪 View experiment at: http://localhost:5000/#/experiments/760520234981114590
🏃 View run secretive-skunk-447 at: http://localhost:5000/#/experiments/760520234981114590/runs/b6d4919e51cc485eb1361d4c49e85cf6
🧪 View experiment at: http://localhost:5000/#/experiments/760520234981114590
🏃 View run intrigued-seal-678 at: http://localhost:5000/#/experiments/760520234981114590/runs/26d5c0eb393f4fbb85c6b9bd843fc92e
🧪 View experiment at: http://localhost:5000/#/experime

2025/05/10 03:46:45 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\RTX\miniconda3\envs\mlops-course\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2025/05/10 03:46:45 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\RTX\miniconda3\envs\mlops-cou

🏃 View run angry-conch-159 at: http://localhost:5000/#/experiments/760520234981114590/runs/0b8484d0ae8041679e8f2c9131cc34c8
🧪 View experiment at: http://localhost:5000/#/experiments/760520234981114590
🏃 View run bemused-lamb-22 at: http://localhost:5000/#/experiments/760520234981114590/runs/99c8549b84f54811949abab645491af1
🧪 View experiment at: http://localhost:5000/#/experiments/760520234981114590
🏃 View run mysterious-carp-595 at: http://localhost:5000/#/experiments/760520234981114590/runs/0ef8ba85bec4475983f1cd59baf00cb0
🧪 View experiment at: http://localhost:5000/#/experiments/760520234981114590
🏃 View run upbeat-boar-376 at: http://localhost:5000/#/experiments/760520234981114590/runs/afbc0a4b371844dcbb0efdab6c77f25b
🧪 View experiment at: http://localhost:5000/#/experiments/760520234981114590
🏃 View run capable-lamb-600 at: http://localhost:5000/#/experiments/760520234981114590/runs/c5381ed6349e4bc09792e878c28893fa
🧪 View experiment at: http://localhost:5000/#/experiments/760520234

2025/05/10 03:46:55 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '55ff1b1dcdc246c0964b215cf785ea89', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2025/05/10 03:46:55 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\RTX\miniconda3\envs\mlops-course\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Mis

🏃 View run delightful-moth-363 at: http://localhost:5000/#/experiments/760520234981114590/runs/3138d388a1f948afb3d266ea5ae2c939
🧪 View experiment at: http://localhost:5000/#/experiments/760520234981114590
🏃 View run wise-mare-284 at: http://localhost:5000/#/experiments/760520234981114590/runs/7f030541e8d545749aedefa4137ddfdf
🧪 View experiment at: http://localhost:5000/#/experiments/760520234981114590
🏃 View run fun-stag-156 at: http://localhost:5000/#/experiments/760520234981114590/runs/ac606bde3430453aa6fad0084f6d105e
🧪 View experiment at: http://localhost:5000/#/experiments/760520234981114590
🏃 View run traveling-goose-87 at: http://localhost:5000/#/experiments/760520234981114590/runs/8a8482339f784e618e0e4be904c8a6a4
🧪 View experiment at: http://localhost:5000/#/experiments/760520234981114590
🏃 View run debonair-goose-605 at: http://localhost:5000/#/experiments/760520234981114590/runs/1f37a4c1d1b84c88ab7b3df503a20f4e
🧪 View experiment at: http://localhost:5000/#/experiments/760520234